# Create Notebooks but not run them

One should check [how-to-run-notebook-on-cluster.ipynb](../how-to-run-notebook-on-cluster.ipynb) for executing the notebooks on a cluster.
For here, we only generate evaluation notebooks **if they do not exist**.
Thus, you can effectively force a rerun, or update the notebook to the latest version, by removing the old notebooks.
You can also choose to remove the JSON data or not, depending on your needs.

In [2]:
import os, sys, time
import papermill as pm
from dotmap import DotMap as dmap
from slugify import slugify

template_path = 'template-simple.ipynb'
inspected = pm.inspect_notebook(template_path)
print("available keys")
for key, value in inspected.items():
    v = dmap(value)
    print(f"    - {key}: {v.inferred_type_name} = {v.default}")

available keys
    - decoder_eval: str = '"mwpm"'
    - code_eval: str = 'f"rsc(d@{c.d},p@{p})"'
    - noise_eval: str = '"none"'
    - config_vec_eval: str = '[dmap(d@3), dmap(d@5), dmap(d@7), dmap(d@9), dmap(d@11)]'
    - p_center: float = 0.01
    - per10_p_count: int = 3
    - slurm_maximum_jobs: None = 360
    - slurm_cores_per_node: int = 18
    - slurm_mem_per_job: int = 4
    - slurm_walltime: str = "11:00:00"
    - slurm_partition: str = "scavenge"
    - local_maximum_jobs: None = 10
    - mark_finished: bool = False


## Rotated Surface Code

In [5]:
d_vec = [3, 5, 7, 9, 11]

rsc_parameters = dict(
    code_eval = 'f"rsc(d@{c.d},p@{p})"',
    noise_eval = '"none"',
    p_center = 0.01,
    config_vec_eval="[" + ",".join(f"dmap(d@{d})" for d in d_vec) + "]",
)

print("config_vec_eval:", rsc_parameters["config_vec_eval"])

folder = slugify(rsc_parameters["code_eval"]) + "_" + slugify(rsc_parameters["noise_eval"])
os.makedirs(folder, exist_ok=True)

decoder_eval_vec = [
    '"mwpm"',
    '"mwpf"',
    '"huf"',
    'f"mwpf(c@{50*c.d})"',  # auto scaling
    '"bposd"',
    '"bposd(max_iter@5)"',
    '"bposd(max_iter@5,osd_order@10,osd_method@osd_e)"',
    '"bpuf"',
    '"bpuf(max_iter@5)"',
]

for decoder_eval in decoder_eval_vec:
    parameters = dict(
        **rsc_parameters,
        decoder_eval=decoder_eval,
    )
    notebook_path = os.path.join(folder, slugify(decoder_eval) + ".ipynb")
    if os.path.exists(notebook_path):
        print("skip existing:", notebook_path)
    else:
        print("parameters:", parameters)
        pm.execute_notebook(
            template_path,
            notebook_path,
            parameters=parameters,
            prepare_only=True,  # do not run the notebook
        )

config_vec_eval: [dmap(d@3),dmap(d@5),dmap(d@7),dmap(d@9),dmap(d@11)]
skip existing: f-rsc-d-c-d-p-p_none/mwpm.ipynb
skip existing: f-rsc-d-c-d-p-p_none/mwpf.ipynb
skip existing: f-rsc-d-c-d-p-p_none/huf.ipynb
skip existing: f-rsc-d-c-d-p-p_none/f-mwpf-c-50-c-d.ipynb
skip existing: f-rsc-d-c-d-p-p_none/bposd.ipynb
skip existing: f-rsc-d-c-d-p-p_none/bposd-max-iter-5.ipynb
skip existing: f-rsc-d-c-d-p-p_none/bposd-max-iter-5-osd-order-10-osd-method-osd-e.ipynb
skip existing: f-rsc-d-c-d-p-p_none/bpuf.ipynb
skip existing: f-rsc-d-c-d-p-p_none/bpuf-max-iter-5.ipynb
